In [143]:
import json
from collections import defaultdict

In [144]:
# Read words in to a dictionary
file = open('AFINN.txt')
word_dict = defaultdict(list)
for line in file.readlines():
    pair = line.rstrip().split("\t")
    word = pair[0]
    score = int(pair[1])
    first_letter = pair[0][0]
    word_dict[first_letter].append((word, score))

In [151]:
for letter in word_dict.keys():
    if letter == 'n':
        print(word_dict[letter])

[('n00b', -2), ('naive', -2), ('nasty', -3), ('natural', 1), ('na茂ve', -2), ('needy', -2), ('negative', -2), ('negativity', -2), ('neglect', -2), ('neglected', -2), ('neglecting', -2), ('neglects', -2), ('nerves', -1), ('nervous', -2), ('nervously', -2), ('nice', 3), ('nifty', 2), ('niggas', -5), ('nigger', -5), ('no', -1), ('no fun', -3), ('noble', 2), ('noisy', -1), ('nonsense', -2), ('noob', -2), ('nosey', -2), ('not good', -2), ('not working', -3), ('notorious', -2), ('novel', 2), ('numb', -1), ('nuts', -3)]


In [77]:
# changing keys of dictionary
word_dict['can t stand'] = word_dict.pop('can\'t stand')


In [78]:

for word in word_dict.keys():
    if '-' in word:
        print(word)

cover-up
made-up
once-in-a-lifetime
self-confident
self-deluded
short-sighted
short-sightedness
son-of-a-bitch


In [79]:
# Load melbGrid
location_list = [] # Store grid information
with open('melbGrid.json', 'r') as f:
    grid_dic = json.load(f)
    for location_dict in grid_dic['features']:
        location_list.append(location_dict['properties'])

In [80]:
# Given coordinate and grid information return a location id
def get_id(coordinate, location_list):
    x = coordinate[0]
    y = coordinate[1]
    for location in location_list:
        if x > location['xmin'] and x <= location['xmax'] \
            and y > location['ymin'] and y <= location['ymax']:
            return location['id']
    return False
print(get_id([145, -37.65], location_list))

B2


In [111]:
# Load coordinate and text
twitter_list = [] # Only sotre tweet in the specified location
with open('smallTwitter.json', 'r', encoding = 'utf-8') as f:
    twitter_dic = json.load(f)

for row in twitter_dic['rows']:
    coordinate = row['value']['geometry']['coordinates']
    id = get_id(coordinate, location_list)
    if id:
        text = row['value']['properties']['text']
        twitter_list.append((id, text))
print(twitter_list[0])

('C2', "I'm at @RivaStkilda in St Kilda, VIC https://t.co/kEbicNcmnP")


In [136]:
# Take a single word and a text as input, return multiple matches of the word in that text as a list. 
import re
def allwords(word, text):
    text = re.sub(r'\W\b', ' ',text)
    print(text)
    return re.findall(r'\b%s\b' % word, text, re.I)

print(allwords('good', "Good guidance goOd! all you ide'as people!"))
    

Good guidance goOd! all you ide as people!
['Good', 'goOd']


In [112]:
# The whole text match
score_dict = {}
for location, text in twitter_list:
    for word in word_dict.keys():
        word_list = allwords(word, text)
        length = len(word_list)
        if length > 0:
            score = length * word_dict[word]
            if location in score_dict:
                score_dict[location] += score
            else:
                score_dict[location] = score
        

KeyboardInterrupt: 

In [113]:
len(word_dict.keys())

2477